In [4]:
import Snapshot_code
import pandas as pd

In [5]:
def transform(df):
    df["hs_max_party_size"] = df["hs_max_party_size"].apply(lambda x: str(int(x)) if x < 5 else "5+" if x >= 5 else x)

    df["dream_Africa_ME"] = df["dream_Africa_ME"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)
    df["dream_Asia"] = df["dream_Asia"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)
    df["dream_Caribbean"] = df["dream_Caribbean"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)
    df["dream_Europe"] = df["dream_Europe"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)
    df["dream_HI"] = df["dream_HI"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)
    df["dream_Mexico"] = df["dream_Mexico"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)
    df["dream_South_America"] = df["dream_South_America"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)
    df["dream_USA"] = df["dream_USA"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)
    df["dream_Italy"] = df["dream_Italy"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)
    df["dream_FR"] = df["dream_FR"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)
    df["dream_South_Africa"] = df["dream_South_Africa"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)
    df["dream_Dubai"] = df["dream_Dubai"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)
    df["dream_island"] = df["dream_island"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)

    return df

In [6]:
# new_df2 = pd.read_excel(r"C:\Users\NahianSiddique\OneDrive - Blend 360\Hilton\Analytical Projects\HGV 2022 VIP Analysis\Data\Model Sample\appended_data_18_19_20221213.xlsx")
new_df2 = pd.read_excel(r"C:\Users\NahianSiddique\OneDrive - Blend 360\Hilton\Analytical Projects\HGV 2022 VIP Analysis\Data\Model Sample\appended_data_21_22_20221213.xlsx")

new_df2 = new_df2.drop(
    columns=[
        "lead_id",
        "t0_baseline_date",
        "full_tour_id",
    ]
)

In [ ]:
# mapping_dict = {"Baseline": "dataset_1", "Segment_1": "dataset_2", "Segment_2": "dataset_3"}
mapping_dict = {"Baseline": "dataset_2", "Segment_1": "dataset_3"}
# mapping_dict = {"Baseline": "dataset_1", "Segment_1": "dataset_4"}

# Set up variables for snapshot
file_name = "profiles_2-3_21-22"
seg_var = "source"
bin_vars_path = "Data/HGV_VIP_attributes_binning.csv"
# Read in file and set bins

In [ ]:
profile = Snapshot_code.Snapshot_Profile(
    profile_data=new_df2,
    segment_var=seg_var,
    continuous_path=bin_vars_path,
    segments=None,
    segment_names=None,
    include=None,
    variable_order=None,
    nbins=5,
    other_segment=False,
    file=file_name,
    exclude=[],
    PPT=True,
    plot_index=False,
    continuous=[],
    excludeother=False,
    mapping_dict=mapping_dict,
)